In [1]:
\l p.q

In [2]:
p)import numpy as np
p)k = np.array(['2007-07', '2006-01', '2010-08'], dtype='datetime64')

In [4]:
k:.p.get[`k]

In [5]:
k[`:dtype.name]`

"datetime64[M]"


In [33]:
p)import datetime
p)k = np.array(['2003-06-28T17:26:01.260806768', '2002-08-17T16:36:35.216906816', '2003-11-07T05:38:53.870536832', '2001-06-26T18:10:48.896862568', '2000-09-11T21:28:21.496423780', '2002-05-11T13:56:52.890104944'],dtype='datetime64[ns]')

In [34]:
print k:.p.get[`k]

['2003-06-28T17:26:01.260806768' '2002-08-17T16:36:35.216906816'
 '2003-11-07T05:38:53.870536832' '2001-06-26T18:10:48.896862568'
 '2000-09-11T21:28:21.496423780' '2002-05-11T13:56:52.890104944']


In [35]:
k[`:dtype.name]`

"datetime64[ns]"


In [40]:
print k:.p.import[`numpy;`:array;.p.eval"list(['2007-07-13', '2006-01-13', '2010-08-13'])";`dtype pykw `datetime64]

['2007-07-13' '2006-01-13' '2010-08-13']


In [12]:
print k[`:dtype.name]

datetime64[D]


In [1]:
encode:{[alist]
    changed:where differ alist;
    : (deltas 1_changed,count alist),'alist changed;
    };

In [2]:
decode:{[pairs]
    :raze {$[1~count x;:x;:(first x)#last x];} each pairs;
    };

In [29]:
p)import cv2
p)img = cv2.imread("/mnt/disks/rockstar/multi_material_detection/data/display/Untitled1.png")
p)print(img.shape)
img:.p.get[`img]


(2865, 2585, 3)


In [26]:
from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout
from skimage.transform import resize

In [9]:
loadimg:.p.import[`keras;`:preprocessing;`:image;`:load_img]
model:.p.import[`keras;`:Model]
es:.p.import[`keras;`:callbacks;`:EarlyStopping]
chkpt:.p.import[`keras;`:callbacks;`:ModelCheckpoint]
rlr:.p.import[`keras;`:callbacks;`:ReduceLROnPlateau]
lm:.p.import[`keras;`:models;`:load_model]
idg:.p.import[`keras;`:preprocessing;`:image;`:ImageDataGenerator]
inp:.p.import[`keras;`:layers;`:Input]
conv2d:.p.import[`keras;`:layers;`:Conv2D]
conv2dt:.p.import[`keras;`:layers;`:Conv2DTranspose]
maxp2d:.p.import[`keras;`:layers;`:MaxPooling2D]
concat:.p.import[`keras;`:layers;`:concatenate]
dropout:.p.import[`keras;`:layers;`:Dropout]
input:.p.import[`keras;`:layers;`:Input]

p)from skimage.transform import resize
resize:.p.get[`resize]

In [8]:
buildmodel:{[inputlayer;startneurons]
    conv1:(conv2d[startneurons*1;(3;3);`activation pykw `relu;`padding pykw `same])[inputlayer];
    conv1:(conv2d[startneurons*1; (3;3); `activation pykw `relu; `padding pykw `same])[conv1];
    pool1:(dropout[0.25])[maxp2d[(2;2)][conv1]];

    / 64 -> 32
    conv2:(conv2d[startneurons*2; (3;3); `activation pykw `relu; `padding pykw `same])[pool1];
    conv2:(conv2d[startneurons*2, (3;3); `activation pykw `relu; `padding pykw `same])[conv2];
    pool2:(dropout[0.25])[maxp2d[(2;2)][conv2]];

/ 64 -> 32
    conv3:conv2d[startneurons * 4; (3;3); `activation pykw `relu; `padding pykw `same][pool2];
    conv3:conv2d[startneurons * 4; (3;3); `activation pykw `relu; `padding pykw `same][conv3];
    pool3:(dropout[0.25])[maxp2d[(2;2)][conv3]];

/ 16 -> 8
    conv4:conv2d[startneurons * 8; (3;3); `activation pykw `relu; `padding pykw `same][pool3];
    conv4:conv2d[startneurons * 8; (3;3); `activation pykw `relu; `padding pykw `same][conv4];
    pool4:(dropout[0.25])[maxp2d[(2;2)][conv4]];

/ Middle
    convm:conv2d[startneurons * 16; (3;3); `activation pykw `relu; `padding pykw `same][pool4];
    convm:conv2d[startneurons * 16; (3;3); `activation pykw `relu; `padding pykw `same][convm];

/ 8 -> 16
    deconv4:conv2dt[startneurons * 8; (3;3);`strides pykw (2;2); padding pykw `same][convm];
    uconv4:concat[deconv4;conv4];
    uconv4:(dropout[0.5])[uconv4];
    uconv4:conv2d[startneurons * 8; (3;3); `activation pykw `relu; `padding pykw `same][uconv4];
    uconv4:conv2d[startneurons * 8; (3;3); `activation pykw `relu; `padding pykw `same][uconv4];

/ 16 -> 32
    deconv3:conv2dt[startneurons * 4;(3;3);`strides pykw (2;2);`padding pykw `same][uconv4];
    uconv3:concatenate[deconv3;conv3];
    uconv3:(dropout[0.5])[uconv3];
    uconv3:conv2d[startneurons * 4;(3;3);`activation pykw `relu;`padding pykw `same][uconv3];
    uconv3:conv2d[startneurons * 4;(3;3);`activation pykw `relu;`padding pykw `same][uconv3];

/ 32 -> 64
    deconv2:conv2dt[startneurons * 2; (3;3), `strides pykw (2;2); `padding pykw `same][uconv3];
    uconv2:concat[deconv2;conv2];
    uconv2:(dropout[0.5])[uconv2];
    uconv2:conv2d[startneurons * 2; (3;3); `activation pykw `relu; `padding pykw `same][uconv2];
    uconv2:conv2d[startneurons * 2; (3;3); `activation pykw `relu; `padding pykw `same][uconv2];

    deconv1:conv2dt[(startneurons * 1); (3;3); `strides pykw (2;2); `padding pykw `same][uconv2];
    uconv1:concat[deconv1;conv1];
    uconv1:(dropout[0.5])[uconv1];
    uconv1:conv2d[startneurons * 1;(3;3); `activation pykw `relu; `padding pykw `same][uconv1];
    uconv1:conv2d[startneurons * 1;(3;3); `activation pykw `relu; `padding pykw `same][uconv1];

    / uconv1 = Dropout(0.5)(uconv1)
    :outputlayer:conv2d[1; (1;1); `padding pykw `same, `activation pykw `sigmoid][uconv1]}


In [9]:
imgsz:128
inputlayer:input[(imgsz;imgsz;1)]
outputlayer:buildmodel[inputlayer;16]

2018-08-30 06:22:46.484416: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-08-30 06:22:47.696138: E tensorflow/core/common_runtime/direct_session.cc:158] Internal: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY; total memory reported: 11996954624


[0;31mcall: Failed to create session.[0m: [0;31mcall: Failed to create session.[0m